In [1]:
import pandas as pd

import numpy as np
#from sklearn.preprocessing import StandardScaler
import pickle


In [2]:
myDataset = pd.read_csv("Health_Risk_Prediction.csv")
df2 = myDataset.copy() 
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()



In [7]:
# ============================
# LOAD MODEL + ENCODERS
# ============================
#model = pickle.load(open("model.sav", "rb"))
load_model = pickle.load(open("DT_Class_FinalModel.sav",'rb'))  #Read Binary from the file 
encoders = pickle.load(open("encoders.sav", "rb"))

print("Loaded model and encoders successfully.")
print("Encoders available:", encoders.keys())


# ============================
# INPUT ORDER (must match training)
# ============================
input_order = [
    "age",
    "bmi",
    "sleep",
    "exercise",
    "sugar_intake",
  #  "profession",
    "smoking",
    "alcohol",
   # "married"
]


# ============================
# ENCODE INPUT VALUES
# ============================
def encode_input(data):
    encoded = []

    for col in input_order:
        value = data[col]

        if col in encoders:
            # Validate
            if value not in encoders[col].classes_:
                raise ValueError(
                    f"ERROR: '{value}' is not a valid label for '{col}'. "
                    f"Allowed: {list(encoders[col].classes_)}"
                )

            encoded_val = encoders[col].transform([value])[0]
        else:
            encoded_val = float(value)

        encoded.append(encoded_val)

    return np.array([encoded])


# ============================
# DECODE OUTPUT LABEL (health_risk)
# ============================
def decode_output(pred):
    pred = [int(pred)]  # convert scalar → 1D array
    return encoders["health_risk"].inverse_transform(pred)[0]


# ============================
# OPTIONAL — DECODE INPUT BACK
# ============================
def decode_input(encoded_array):
    decoded = {}

    for i, col in enumerate(input_order):
        value = encoded_array[i]

        if col in encoders:
            decoded[col] = encoders[col].inverse_transform([int(value)])[0]
        else:
            decoded[col] = value

    return decoded


# ============================
# MAIN PREDICTION FLOW
# ============================
def predict_user(data):
    print("\n===== INPUT RECEIVED =====")
    print(data)

    # Encode user input
    encoded_x = encode_input(data)
    print("\nEncoded Input:", encoded_x)

    # Predict
    #pred = model.predict(encoded_x)[0]
    pred = load_model.predict(encoded_x)[0]
    print("\nEncoded Prediction:", pred)

    # Decode output
    decoded_result = decode_output(pred)
    print("\nDecoded Health Risk:", decoded_result)

    # Decode input (optional)
    decoded_input = decode_input(encoded_x[0])
    print("\nDecoded Input:", decoded_input)

    print("\n===== FINAL RESULT =====")
    return decoded_result


# ============================
# EXAMPLE USAGE
# ============================
#if __name__ == "__main__":

    # Example user input in string form (same as UI/Frontend)

user_data = {
    "age": 35,
    "bmi": 26.5,
    "sleep": 5.5,
    "exercise": "low",
    "sugar_intake": "low",
    "smoking": "no",
    "alcohol": "yes"
}

user_data1 = {
    "age": 65,
    "bmi": 36.6,
    "sleep": 4.5,
    "exercise": "medium",
    "sugar_intake": "high",
    "smoking": "yes",
    "alcohol": "yes"
}


Loaded model and encoders successfully.
Encoders available: dict_keys(['exercise', 'sugar_intake', 'smoking', 'alcohol', 'health_risk'])


C:\Anaconda\anaconda3\envs\v_env1\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.6.1 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Anaconda\anaconda3\envs\v_env1\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator GridSearchCV from version 1.6.1 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
result = predict_user(user_data)


print("\nPredicted Health Risk:", result)


===== INPUT RECEIVED =====
{'age': 35, 'bmi': 26.5, 'sleep': 5.5, 'exercise': 'low', 'sugar_intake': 'low', 'smoking': 'no', 'alcohol': 'yes'}

Encoded Input: [[35.  26.5  5.5  1.   1.   0.   1. ]]

Encoded Prediction: 1

Decoded Health Risk: low

Decoded Input: {'age': 35.0, 'bmi': 26.5, 'sleep': 5.5, 'exercise': 'low', 'sugar_intake': 'low', 'smoking': 'no', 'alcohol': 'yes'}

===== FINAL RESULT =====

Predicted Health Risk: low


C:\Anaconda\anaconda3\envs\v_env1\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


In [8]:
result1 = predict_user(user_data1)


print("\nPredicted Health Risk:", result1)


===== INPUT RECEIVED =====
{'age': 65, 'bmi': 36.6, 'sleep': 4.5, 'exercise': 'medium', 'sugar_intake': 'high', 'smoking': 'yes', 'alcohol': 'yes'}

Encoded Input: [[65.  36.6  4.5  2.   0.   1.   1. ]]

Encoded Prediction: 0

Decoded Health Risk: high

Decoded Input: {'age': 65.0, 'bmi': 36.6, 'sleep': 4.5, 'exercise': 'medium', 'sugar_intake': 'high', 'smoking': 'yes', 'alcohol': 'yes'}

===== FINAL RESULT =====

Predicted Health Risk: high


C:\Anaconda\anaconda3\envs\v_env1\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
